# Session 06 - Reciprocal Best BLAST Hits (RBBH) <img src="data/JHI_STRAP_Web.png" style="width: 150px; float: right;"> 

## Learning Outcomes

* Conduct `BLASTP` comparisons between protein complements for prokaryotes
* Identify reciprocal best `BLAST` matches
* Visualise and interpret reciprocal best `BLAST` matches using a range of tools

## Introduction

### Reciprocal best `BLAST` hits

As discussed in the lecture, reciprocal Best `BLAST` Hits (RBBH) are a common approximation to orthology for protein sequences, used in comparative genomics. An RBBH is found when two proteins, each in a different genome, find each other as the best scoring match in the other genome.

### Python code

We will use the [`Biopython`](http://www.biopython.org) libraries to interact with and manipulate sequence data, and the [`Pandas`](http://pandas.pydata.org/) data analysis libraries to manipulate numerical data and `BLAST` output.

Most of the code you will use is imported from the local `bs32010` module in this directory, to avoid clutter in this notebook. You can inspect this module if you are interested in how the functions work.

In [3]:
%pylab inline

from bs32010 import ex06  # Local functions and data

Populating the interactive namespace from numpy and matplotlib


SyntaxError: invalid syntax (ex06.py, line 157)